<a href="https://colab.research.google.com/github/Kimiko00/Training-GSP/blob/main/new_gsp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy
import re
import pandas as pd
from collections import defaultdict as de

In [ ]:
########################### READ ME #################################

# ganti beberapa variable pada

## MEMAKAI DATA SUPER STORE
# df1 = pd.read_csv('.csv') ==> sesuaikan dengan file .csv yang ingin dibaca (file ada didalam alenia atasnya)

## FUNGSI GSP
# for i in dataset['Sub-Category'].values: ==> pada ['...'] sesuaikan pada colom didalam data.csv

## MAIN
# for i in dataset['Sub-Category'].values: ==> pada ['...'] sesuaikan pada colom didalam data.csv
# di=gps_de(r_1_di,2) ==> pada (r_1_di, ...) sesuaikan angka dibelakang dengan panjang sequence yang ingin dicari
# r_1_di[j[0]]+=1 # membaca 1 huruf didepan setiap data ==> hapus +=1 jika ingin nama product ditampilkan

# **MEMAKAI DATA SUPER STORE**

In [ ]:
# membaca file xls
xls = pd.ExcelFile('Superstore_Data_One_Region_Per_Sheet.xlsx')

In [ ]:
# membaca file excel per sheetnya 
West_Superstore = pd.read_excel(xls, 'West_Superstore')
Central_Superstore = pd.read_excel(xls, 'Central_Superstore')
East_Superstore = pd.read_excel(xls, 'East_Superstore')
South_Superstore = pd.read_excel(xls, 'South_Superstore')

# menyimpan per sheet name sebagai file csv yang berbeda
West_Superstore = West_Superstore.to_csv('West_Superstore.csv')
Central_Superstore = Central_Superstore.to_csv('Central_Superstore.csv')
East_Superstore = East_Superstore.to_csv('East_Superstore.csv')
South_Superstore = South_Superstore.to_csv('South_Superstore.csv')

In [ ]:
df1 = pd.read_csv('West_Superstore.csv', delimiter=',')
df1.replace(',','', regex=True, inplace=True)
df1 = df1[['Customer ID', 'Product Name']] # menggunakan hanya kolom customer id dan sub-category

# fungsi group by dan menggabungkan semua pesanan per customer id menjadi deret 
df1 = df1.groupby('Customer ID')[['Product Name']].aggregate(lambda x: ",".join(list(x))).reset_index()

#menyimpan file kedalam bentuk .csv baru
df1.to_csv('data.csv')

# **FUNGSI GSP**

In [ ]:
# fungsi untuk menghapus sebuah item set didata
def dell(d,na):
    for i in na:
        del d[i]
    return d

# funsgi untuk menambahkan item ke dalam sequence 
# jika V (value secuence) lebih besar dari value yang dicari(n)    
def gps_de(d,n):
    name=[]
    for k,v in d.items(): # untuk setiam item didalam sequence dan valuenya
        if v<n:
            name.append(k) # dimasukkan ke dalam jenis sequence baru ke variable nama
    return dell(d,name)

# fungsi prunning
def gps_cr(d):
    name=[]
    for k,v in d.items():
        name.append(k)
    name_new=[i+j[-1] for i in name for j in name if i[1:]==j[:-1]] #prunning
    return name_new

# fungi pencarian candidat
def gps_crdi(name,dataset):
    r=de(int)
    for i in name:
        b=''
        for j in range(len(i)-1):
            b+=i[j]+'.*'
        b+=i[-1]
        # penarikan data didalam dataset
        for i in dataset['Product Name'].values:
            i_new=i.replace(',','') # mengganti , didalam data: (input) A,B --> AB (output)
            res=re.search(b,i_new) # ditampung
            if not(res==None):
                r[b.replace('.*','')]+=1
    return r

# **MAIN**

In [ ]:
# data reading
with open('data.csv')as f:
    dataset=pd.read_csv(f)
    r_1=[]
    r_1_di=de(int)
    for i in dataset['Product Name'].values:
        for j in set(i.split(',')): # split item berdasarkan koma
            r_1_di[j]+=1 # membaca 1 huruf didepan setiap data
    
    # Sequence pertama
    di=gps_de(r_1_di,3) # 2 adalah besaran min yang dicari atau ditetapkan
    print('L1:')
    print(di)

    # candidat kedua
    name=[]
    for k,v in di.items():
        name.append(k)

    name_new=[i+j for i in name for j in name]
    r_di=gps_crdi(name_new,dataset) # candidat
    di_3=gps_de(r_di,3) # pruning
    nuu=1

    # looping untuk mecari setiap sequence dan candidat 
    # yang ada selama panjang dari sequence pertama tidak 0
    while(len(di_3)!=0):
        print('L'+str(nuu+1)+':')
        print(di_3)
        # mencari confident setiap sequence
        for k,v in di_3.items():
          print("Support Count {}: {}%".format(k, v/len(k)*100))
        # pruning dan candidat selanjutnya
        name_n=gps_cr(di_3) # candidat selanjutnya
        r_di=gps_crdi(name_n,dataset) # pruning selanjutnya
        di_3=gps_de(r_di,2)
        nuu+=1 # increment untuk L+ (penamaan per run)